In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>")) # makes the notebook fill the whole window

import numpy as np
import pandas as pd
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure

# from bokeh.io import output_file, show
# from bokeh.models import ColumnDataSource, GMapOptions
# from bokeh.plotting import gmap

from PIL import Image, ImageDraw
import os
import shutil

from mpl_toolkits.basemap import Basemap

%matplotlib inline

#Turn off interactive plotting for pyplot
plt.ioff()

ModuleNotFoundError: No module named 'matplotlib.colorbar'

In [ ]:
# Read in a particular date
date = "20170901"
df = pd.read_csv('./WWLN_2017/A' + date + '.loc',header=None,names=["Date","Time","Latitude","Longitude","Error","Number of Stations"],low_memory=False)

#Functions to get the time
def GetHours(x):
    hours = int(x[0:2])
    return hours

def GetMinutes(x):
    minutes = int(x[3:5])
    return minutes

def GetSeconds(x):
    seconds = int(x[6:8])
    return seconds

def GetMicroseconds(x):
    ms = int(x[9:])
    return ms

def GetTotalMinute(x):
    hours = int(x[0:2])
    minutes = int(x[3:5])
    return 60 * hours + minutes

df['Hours'] = df['Time'].apply(GetHours)
df['Minutes'] = df['Time'].apply(GetMinutes)
df['Seconds'] = df['Time'].apply(GetSeconds)
df['MicroSeconds'] = df['Time'].apply(GetMicroseconds)
df['TotalMinutes'] = df['Time'].apply(GetTotalMinute)

In [ ]:
# Make a directory to store the images
os.mkdir('./data/images')

In [ ]:
plt.figure(figsize=(30,15))
# Map of the world
m = Basemap(llcrnrlon=-180, llcrnrlat=-90,urcrnrlon=180,urcrnrlat=90,lon_0=0,lat_0=0)
m.drawmapboundary(fill_color='#A6CAE0', linewidth=0)
m.fillcontinents(color='grey', alpha=0.7, lake_color='grey')
m.drawcoastlines(linewidth=0.1, color="white")
 
# Plot the lightning data
m.plot(df[df['TotalMinutes'] <= 30]['Longitude'], df[df['TotalMinutes'] <= 30]['Latitude'], linestyle='none', marker="o", markersize=8, alpha=0.7, c="orange", markeredgecolor="black", markeredgewidth=1)
plt.title("0 minutes")
plt.savefig("./data/test",bbox_inches='tight')

In [ ]:
images = []

#Turn off the max figure warnjing for MatPlotLib
mpl.rcParams.update({'figure.max_open_warning': 0})

#Every 30 minutes, plot the lightning data
for x in range(0,1440,30):#0,1440,30
    plt.figure(figsize=(30,15))
    # Map of the world
    m = Basemap(llcrnrlon=-180, llcrnrlat=-90,urcrnrlon=180,urcrnrlat=90,lon_0=0,lat_0=0)
    m.drawmapboundary(fill_color='#A6CAE0', linewidth=0)
    m.fillcontinents(color='grey', alpha=0.7, lake_color='grey')
    m.drawcoastlines(linewidth=0.1, color="white")

    # Plot the lightning data
    m.plot(df[(df['TotalMinutes'] <= x + 30) & (df['TotalMinutes'] >= x)]['Longitude'], df[(df['TotalMinutes'] <= x + 30) & (df['TotalMinutes'] >= x)]['Latitude'], linestyle='none', marker="o", markersize=8, alpha=0.7, c="orange", markeredgecolor="black", markeredgewidth=1)
    plt.title(str(x) + "minutes")
    name = "./data/images/Minute" + str(x)
    plt.savefig(name,bbox_inches='tight')
    #store images for gif creation
    images.append(Image.open(name + '.png'))

#create gif
images[0].save('./data/gifs/'+ date + '.gif', save_all=True, append_images=images[1:], optimize=False, duration=100, loop=0)

In [ ]:
shutil.rmtree('./data/images')

In [ ]:
#Import Fuzzy C-Means, put it on the data, run it, plot it
# VERY SLOW
import FCM

test = FCM.FCM(df[df['TotalMinutes'] <= 20][['Latitude','Longitude']],15,2,maxiter=5,genCentroids=True)

test.fit()

test.plot()

In [ ]:
test._A[0].sum()